In [1]:
from transformers import AutoModelForImageTextToText, AutoProcessor
import torch
model = AutoModelForImageTextToText.from_pretrained(
    "Hcompany/Holo1-7B",
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    device_map="auto",
)

processor = AutoProcessor.from_pretrained("Hcompany/Holo1-7B")


/home/xd/Documents/ML_AI/android-automation-tester/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.21s/it]
Some parameters are on the meta device because they were offloaded to the cpu.
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back w

In [2]:
# navigation = navigation.NavigationStep(**json.loads(navigation_str))
# print(navigation)
# Expected NavigationStep(note='', thought='I need to select the check-out date as August 3rd and then proceed to search for hotels.', action=ClickElementAction(action='click_element', element='August 3rd on the calendar', x=777, y=282))

In [3]:
# from PIL import Image
# from transformers.models.qwen2_vl.image_processing_qwen2_vl import smart_resize
# image_url = "/home/xd/Documents/ML_AI/automation_modeling/test1.jpeg"
# image = Image.open(image_url)  # Load as PIL.Image

# image_processor = processor.image_processor

# Resize image using smart_resize
# resized_height, resized_width = smart_resize(
#     image.height,
#     image.width,
#     factor=image_processor.patch_size * image_processor.merge_size,
#     min_pixels=image_processor.min_pixels,
#     max_pixels=image_processor.max_pixels,
# )

# image = image.resize(size=(resized_width, resized_height), resample=Image.BICUBIC)


In [3]:
from PIL import Image
image_url = "/home/xd/Documents/ML_AI/android-automation-tester/images/resized_portrait.jpg" 
image = Image.open(image_url)  # Load as PIL.Image


# guidelines = "Localize an element on the GUI image according to my instructions and output a click position as Click(x, y) with x num pixels from the left edge and y num pixels from the top edge. The x, y must be exactly in the center of the GUI element"
# instruction = "Select app icon battery percentage"
# messages = [
#         {
#             "role": "user",
#             "content": [
#                 {
#                     "type": "image",
#                     "url": image_url,
#                 },
#                 {"type": "text", "text": f"{guidelines}\n{instruction}"},
#             ],
#         }
#     ]

# inputs = processor.apply_chat_template(
#     messages,
#     tokenize=True,
#     add_generation_prompt=True,
#     return_tensors="pt",
#     return_dict=True,
# ).to(model.device)

import localization
from typing import Any


# Helper function to run inference
def run_inference(messages: list[dict[str, Any]]) -> str:
    # Preparation for inference
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = processor(
        text=[text],
        images=image,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to("cuda")

    generated_ids = model.generate(**inputs, max_new_tokens=128)
    generated_ids_trimmed = [out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)]
    return processor.batch_decode(generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False)

instruction = "Select tide app"
prompt = localization.get_localization_prompt(image, instruction)
coordinates = run_inference(prompt)[0]
print(coordinates)
# Expected Click(352, 348)


Click(275, 814)


In [ ]:
# generated_ids = model.generate(**inputs, max_new_tokens=128)

# decoded = processor.batch_decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)

# Click(352, 348)


In [ ]:
# decoded